In [1]:
import os
import numpy as np
import random
import glob
from datetime import datetime
import logging
import sys
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.backend import clear_session
import gc
from sklearn.model_selection import train_test_split

2024-12-18 15:56:15.218078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 15:56:15.335404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 15:56:15.366562: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 15:56:15.583507: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-18 15:56:17.027013: W tensorflow/compiler/tf2

In [2]:
# Limpa a sessão antes de construir o modelo
clear_session()

# Configurações de GPU
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Define a política de precisão mista
set_global_policy('mixed_float16')

I0000 00:00:1734548179.599239   18305 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734548179.823028   18305 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734548179.823093   18305 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
output_path = "modelos" #caminha onde é salvo o modelo
files_per_batch = 1000
labeled_data_dir = "train_graphs" #caminho dos dados para treinar o modelo
param_v_a_1 = [4, 3, 2] #camadas compartilhadas rede bound
param_v_a_2 = [3, 2, 2] #camadas ocultas
param_p_a_1 = [6, 4, 3] #camadas compartilhadas rede brach
param_p_a_2 = [9, 6, 2] #camadas ocultas
param_p_b = 250 #batch size
param_v_b =  250
param_p_l = 0.001 #taxa de aprendizado
param_v_l = 0.001
tam = 150
use_value_model = True #se vai treinar a rede de bound

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [4]:
# Pegando todos os arquivos do diretório
files = sorted([os.path.basename(ii) for ii in glob.glob(f"{labeled_data_dir}/*.dimacs")])
files_treino, files_teste = train_test_split(files, test_size=0.1, random_state=42)
files_treino = files_treino[:3000]
files_t, files_v = train_test_split(files_treino, test_size=0.2, random_state=42)

In [5]:
def parse_file_pointer(fp):
    lines = [ll.strip() for ll in fp]
    ii = 0
    labels = []
    res = []
    cli = []
    numLinhas = 0
    while ii < len(lines):
        line = lines[ii]
        #contando o numero de vertices do grafo
        if "cliqueatual" not in line:
            ii += 1
            numLinhas += 1
            continue

        #pegando a clique atual
        if ii+1 >= len(lines):
            break
        line = line[3:]
        spritado = line.split()
        clique = [int(elem) for elem in spritado[1:]]
        if(numLinhas < tam):
            dif = tam - numLinhas
            clique.extend([0]*dif)
        cli.append(clique)

        #criando o vetor de movimento
        line = lines[ii+1]
        sp = line.split()
        mv = int(sp[-1])
        label = [0] * tam
        label[mv] = 1
        labels.append(label)

        #lendo o grafo
        cells = []
        for tt in range(numLinhas, 0, -1):
            cell_line = lines[ii - tt][3:]
            cells.extend([int(float(cc)) for cc in cell_line.split(", ")])
            if(numLinhas < tam):
                dif = tam - numLinhas
                cells.extend([0]*dif)
        while len(cells) < tam * tam:
            cells.extend([0]*tam)
        res.append(cells)
        ii += (numLinhas+2)
    labels_v = list(range(len(labels),0, -1))
    return (res, cli, labels, labels_v)

In [6]:
def estruturar_entrada(batch_input, batch_labels):
    # Dividir a entrada em uma lista de 151 tensores de forma (batch_size, tam)
    batch_input_list = [batch_input[:, i, :] for i in range(batch_input.shape[1])]
            
    # Converter para tensores do TensorFlow
    x_batch = [tf.convert_to_tensor(tensor, dtype=tf.float32) for tensor in batch_input_list]
    input_dict = {f'input_{i}': tensor for i, tensor in enumerate(x_batch)}
    y_batch = np.array(batch_labels)
    return input_dict, y_batch

In [7]:
def combinar_entrada(res, clique, labels, remaining_batch_input=[], remaining_batch_labels=[]):
    combined_input = np.array([np.hsplit(np.concatenate([res[i], clique[i]]), tam + 1) for i in range(len(clique))])
    if len(remaining_batch_input) != 0:
        combined_input = np.concatenate((remaining_batch_input, combined_input), axis=0)
        labels = remaining_batch_labels + labels
    return combined_input, labels

In [8]:
def parse_dir(files):
    res = []
    cli = []
    labels = []
    labels_v = []
    random.seed(42)
    random.shuffle(files)
    random.seed()
    for ff in files:
        with open(os.path.join(labeled_data_dir,ff), 'r') as fp:
            rr, cc, ll, ll_v = parse_file_pointer(fp)
            res.extend(rr)
            cli.extend(cc)
            labels.extend(ll)
            labels_v.extend(ll_v)
    return res, cli, labels, labels_v

In [9]:
def ler_arquivos(batch_files, value):
    res, clique, labels, labels_v = parse_dir(batch_files)
    if value:
        labels = labels_v
    return res, clique, labels

In [10]:
def data_generator(batch_size, value):
    # Gerador de treinamento
    def create_generator(file_list):
        def generator():
            idx = 0
            remaining_batch_input = remaining_batch_labels = []
            while idx < len(file_list):
                batch_files = file_list[idx: idx + files_per_batch]
                res, clique, labels = ler_arquivos(batch_files, value)
                combined_input, labels = combinar_entrada(res, clique, labels, remaining_batch_input, remaining_batch_labels)
                remaining_batch_input = remaining_batch_labels = []
                num_samples = len(combined_input)
                num_batches = num_samples // batch_size
                for batch_idx in range(num_batches):
                    batch_input = combined_input[batch_idx * batch_size: (batch_idx + 1) * batch_size]
                    batch_labels = labels[batch_idx * batch_size: (batch_idx + 1) * batch_size]
                    x_batch, y_batch = estruturar_entrada(batch_input, batch_labels)
                    yield x_batch, y_batch
                idx += files_per_batch
                remaining_samples = num_samples % batch_size
                if remaining_samples > 0:
                    remaining_batch_input = combined_input[-remaining_samples:]
                    remaining_batch_labels = labels[-remaining_samples:]
        return generator
    
    #ler o dataset de validação todo de uma vez ao invés de usar um gerador pra ele
    res, clique, labels = ler_arquivos(files_v, value)
    combined_input, labels = combinar_entrada(res, clique, labels)
    x_value, y_value = estruturar_entrada(combined_input, labels)
    
    return create_generator(files_t)(), x_value, y_value

In [11]:
def output_signature_type_branch(batch_size):
    output_signature=(
            {f"input_{i}": tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32) for i in range(tam + 1)},  # Entradas
            tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32)  # Rótulos
    )
    return output_signature

def output_signature_type_bound(batch_size):
    output_signature=(
            {f"input_{i}": tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32) for i in range(tam + 1)},  # Entradas
            tf.TensorSpec(shape=(batch_size,), dtype=tf.float32)  # Rótulos
    )
    return output_signature

In [12]:
class printbatch(callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        logging.info("Epoch: "+ str(epoch))
    def on_epoch_end(self, epoch, logs={}):
        logging.info(logs)

In [13]:
class LoggerWriter:
    def __init__(self, level):
        self.level = level

    def write(self, message):
        if message != '\n':
            self.level(message)

    def flush(self):
        self.level(sys.stderr)

In [14]:
def learn():
    # Definir camadas de entrada
    inputArray = [Input(shape=(tam,), name=f"input_{i}") for i in range(tam + 1)]

    # Camadas densas compartilhadas
    layer = inputArray
    for i in range(len(param_p_a_1)):
        shared_dense = Dense(tam * param_p_a_1[i], activation='relu')
        layer = [shared_dense(l) for l in layer]

    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(layer)

    #camadas internas
    layer = merged_vector
    for i in range(len(param_p_a_2)):
        layer = Dense((tam+1)*param_p_a_2[i],activation='relu')(layer)

    #camada de saida
    output_layer = Dense(tam, activation='softmax')(layer)

    #compilar modelo
    model = Model(inputs=inputArray, outputs=output_layer)
    adam = optimizers.Adam(learning_rate=param_p_l)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Criar os datasets de treino e validação
    train_generator_func, val_x, val_y = data_generator(param_p_b, False)
    train_dataset = tf.data.Dataset.from_generator(lambda : train_generator_func, output_signature=output_signature_type_branch(param_p_b))

    # Prefetch para melhorar a performance
    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
    
    #treinar modelo
    now = datetime.now()
    model.fit(train_dataset, epochs=5, verbose=1, validation_data=(val_x, val_y),
              callbacks=[printbatch(), EarlyStopping(monitor='val_loss', patience=50, verbose=0),
                         ModelCheckpoint(os.path.join(output_path, "models", "dnn_model_" + str(tam) + "_" + str(now.day) + "." +
                                         str(now.month) + "." + str(now.year) + "_" + "_{epoch:02d}-{val_loss:.2f}.keras"),
                                         monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')])
    model.save(os.path.join(output_path,"dnn_model_" + str(tam) + "_" + str(now.day) + "." +str(now.month) + "." + str(now.year) + "_.keras"))
    del model, val_x, val_y, train_dataset, train_generator_func

In [15]:
def learn_value():
    # Definir camadas de entrada
    inputArray = [Input(shape=(tam,), name=f"input_{i}") for i in range(tam + 1)]

    # Camadas densas compartilhadas
    layer = inputArray
    for i in range(len(param_v_a_1)):
        shared_dense = Dense(tam * param_v_a_1[i], activation='relu')
        layer = [shared_dense(l) for l in layer]

    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(layer)

    #camadas internas
    layer = merged_vector
    for i in range(len(param_v_a_2)):
        layer = Dense(tam*param_v_a_2[i],activation='relu')(layer)

    #camada de saida
    output_layer = Dense(1)(layer)

    #compilar modelo
    model = Model(inputs=inputArray, outputs=output_layer)
    adam = optimizers.Adam(learning_rate=param_v_l)
    model.compile(optimizer=adam, loss='mse', metrics=['mae'])

    # Criar os datasets de treino e validação
    train_generator_func, val_x, val_y = data_generator(param_v_b, True)
    train_dataset = tf.data.Dataset.from_generator(lambda : train_generator_func, output_signature=output_signature_type_bound(param_v_b))

    # Prefetch para melhorar a performance
    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

    #treinar modelo
    now = datetime.now()
    model.fit(train_dataset, epochs=5, verbose=1, validation_data=(val_x, val_y),
              callbacks=[printbatch(),
                         EarlyStopping(monitor='val_loss', patience=50, verbose=0),
                         ModelCheckpoint(os.path.join(output_path, "models", "dnn_value_model_" + str(tam) + "x" + "_" + str(now.day) + "." +
                                        str(now.month) + "." + str(now.year) + "_" + "_{epoch:02d}-{val_loss:.2f}.keras"), monitor='val_loss',
                                         verbose=0, save_best_only=True, save_weights_only=False, mode='auto')])
    model.save(os.path.join(output_path,"dnn_value_model_" +str(tam) + "_" +str(now.day) + "." +str(now.month) +"."+str(now.year)+"_.keras"))
    del model, val_x, val_y, train_dataset, train_generator_func

In [16]:
def main():
    #treinar rede branch
    learn()
    print("rede 1 treinada")
    clear_session()

    gc.collect()
    #treinar rede bound
    if use_value_model:
        learn_value()
        print("rede 2 treinada")
        clear_session()
        gc.collect()

main()

I0000 00:00:1734548180.397687   18305 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734548180.397846   18305 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734548180.397900   18305 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734548180.644603   18305 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734548180.644731   18305 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-18

Epoch 1/5


/home/igor/anaconda3/envs/tcc/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_0', 'input_1', 'input_2', 'input_3', 'input_4', 'input_5', 'input_6', 'input_7', 'input_8', 'input_9', 'input_10', 'input_11', 'input_12', 'input_13', 'input_14', 'input_15', 'input_16', 'input_17', 'input_18', 'input_19', 'input_20', 'input_21', 'input_22', 'input_23', 'input_24', 'input_25', 'input_26', 'input_27', 'input_28', 'input_29', 'input_30', 'input_31', 'input_32', 'input_33', 'input_34', 'input_35', 'input_36', 'input_37', 'input_38', 'input_39', 'input_40', 'input_41', 'input_42', 'input_43', 'input_44', 'input_45', 'input_46', 'input_47', 'input_48', 'input_49', 'input_50', 'input_51', 'input_52', 'input_53', 'input_54', 'input_55', 'input_56', 'input_57', 'input_58', 'input_59', 'input_60', 'input_61', 'input_62', 'input_63', 'input_64', 'input_65', 'input_66', 'input_67', 'input_68', 'input_69'

      1/Unknown 84s 84s/step - accuracy: 0.0080 - loss: 5.0109

I0000 00:00:1734548289.087098   19187 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 13.2984 - val_mae: 2.7588
Epoch 4/5


2024-12-18 16:03:21.468262: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4985357247527059487
2024-12-18 16:03:21.468328: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14129316206994391813
2024-12-18 16:03:21.468342: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8926734945505543141
2024-12-18 16:03:21.468352: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8543878768399353527
2024-12-18 16:03:21.468364: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 7567484325667407703
2024-12-18 16:03:21.468372: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15611418022365529991
2024-12-18 16:03:21.468380: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv i

100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 13.2984 - val_mae: 2.7588
Epoch 5/5


2024-12-18 16:03:25.634547: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4985357247527059487
2024-12-18 16:03:25.634630: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14129316206994391813
2024-12-18 16:03:25.634655: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8926734945505543141
2024-12-18 16:03:25.634672: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8543878768399353527
2024-12-18 16:03:25.634686: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 7567484325667407703
2024-12-18 16:03:25.634699: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15611418022365529991
2024-12-18 16:03:25.634711: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv i

100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 13.2984 - val_mae: 2.7588
rede 2 treinada
